<h2> COMP4528 Lab6 </h2>

<p><b>(Acknowledgement: Lab material courtesy of Professor. Du Huynh of UWA).</b></p>

<p>In this lab, we will be working on the camera calibration task.</p>

<p>Four images, stereo2012a.jpg, stereo2012b.jpg, stereo2012c.jpg, and stereo2012d.jpg, are given for this lab. These images are different views of a calibration target and some objects. For example, the diagram below is stereo2012a.jpg with some text superimposed onto it:</p>

<p align="center">
  <img src="notebook_assets/1.png">
</p>

<p><b>On the calibration target there are 3 mutually orthogonal faces. The points marked on each face form a regular grid. They are all 7cm apart.</b></p>

<p>From the 4 supplied images (stereo2012a.jpg, stereo2012b.jpg, stereo2012c.jpg, and stereo2012d.jpg), <font color="red">choose any image to work on</font>. Use the suggested right-hand coordinate system shown in the diagram above and choose a sufficient number of calibration points on the calibration target.</p>

<p>Store the XYZ coordinates in a file so that you can load the data into Python and use them again and again. Note that each image can be calibrated independently, so you can choose different calibration points to calibrate each image. Neither do the numbers of calibration points need to be the same for your chosen images. <font color="red">Read through the choose_points.py script in the lab materials, understand the code, run the script to generate the XYZ and uv coordinates and store them in the data folder.</font></p>

<p>After the above operation, the variable uv should be a 12 × 2 matrix, each row of which should contain the coordinates of one image point.</p>

<p><font color="red"> Note: You need to ensure that, for each image, the numbers of 3D and 2D calibration points are the same. For example, if your uv variable is a 12 × 2 matrix, then your XYZ variable should be a 12 × 3 matrix. Also, the points should appear in the same order in these two matrices. </font></p>

<p>Use the DLT algorithm to solve the unknown camera calibration matrix of size 3x4. (Refer to lecture slides and textbook: Multiple View Geometry in Computer Vision Section 7 (page 178)).</p>


<h3> Hints: </h3>

<ol>
  <li>In writing your code you may need to "reshape" a 2D vector into a 1D vector. You can use the function np.reshape to reshape a matrix to arbitrary dimensions.</li>
  <li>You can save your calibration matrices using NumPy save command. Read the following links for more details.
  <ul>
    <li><a href="url">https://numpy.org/doc/stable/reference/generated/numpy.save.html</a></li>
    <li><a href="url">https://numpy.org/doc/stable/reference/generated/numpy.load.html</a></li>
  </ul>
  </li>
</ol>

### Import libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


### Utility functions (you don't have to understand implementation details)

In [ ]:
'''
VGG_KR_FROM_P Extract K, R from camera matrix.

[K,R,t] = VGG_KR_FROM_P(P [,noscale]) finds K, R, t such that P = K*R*[eye(3) -t].
It is det(R)==1.
K is scaled so that K(3,3)==1 and K(1,1)>0. Optional parameter noscale prevents this.

Works also generally for any P of size N-by-(N+1).
Works also for P of size N-by-N, then t is not computed.

Original Author: Andrew Fitzgibbon <awf@robots.ox.ac.uk> and awf
Date: 15 May 98

Modified by Shu.
Date: 8 May 20
'''


def vgg_rq(S):
    S = S.T
    [Q, U] = np.linalg.qr(S[::-1, ::-1], mode='complete')

    Q = Q.T
    Q = Q[::-1, ::-1]
    U = U.T
    U = U[::-1, ::-1]
    if np.linalg.det(Q) < 0:
        U[:, 0] = -U[:, 0]
        Q[0, :] = -Q[0, :]
    return U, Q


def vgg_KR_from_P(P, noscale=True):
    N = P.shape[0]
    H = P[:, 0:N]
    print(N, '|', H)
    [K, R] = vgg_rq(H)
    if noscale:
        K = K / K[N-1, N-1]
        if K[0, 0] < 0:
            D = np.diag([-1, -1, np.ones([1, N-2])])
            K = K @ D
            R = D @ R

            test = K*R
            assert (test/test[0, 0] - H/H[0, 0]).all() <= 1e-07

    t = np.linalg.inv(-P[:, 0:N]) @ P[:, -1]
    return K, R, t


### Load the data from saved npy files

In [ ]:
# TODO: Read the image and load saved coordinates npy files
xyz = None
uv = None
img = None


### XYZ normalisation matrix

The normalisation matrix for XYZ coordinates $\mathbf{S}_{\text{norm}}$ is defined as $$\mathbf{S}_{\text{norm}} = \begin{bmatrix}\mathbf{V}\text{diag}(\lambda_1^{-1}, \lambda_2^{-1}, \lambda_3^{-1}) \mathbf{V}^{-1} & -\mathbf{V}\text{diag}(\lambda_1^{-1}, \lambda_2^{-1}, \lambda_3^{-1}) \mathbf{V}^{-1}\mu_\mathbf{x}\\ 0 & 1 \end{bmatrix}$$

where $$\mathbf{V}\text{diag}(\lambda_1, \lambda_2, \lambda_3) \mathbf{V}^{-1} = \text{eig}(\sum_i(\mathbf{X}_{i,\text{nonhom}} - \mu_\mathbf{x})^\top(\mathbf{X}_{i,\text{nonhom}} - \mu_\mathbf{x}))$$
$$\mathbf{X}_{\text{nonhom}} = \begin{bmatrix}x_1 & y_1 & z_1\\ x_2 & y_2 & z_2\\ \vdots & \vdots & \vdots\\ x_N & y_N & z_N\\  \end{bmatrix}$$
$$\mu_\mathbf{x} = \begin{bmatrix}\frac{1}{N}\sum_ix_i & \frac{1}{N}\sum_iy_i & \frac{1}{N}\sum_iz_i\end{bmatrix}$$

In [ ]:
# Generate the normalisation matrix S for XYZ coordinate
def xyz_normalisation_mat(xyz):
    # TODO: Complete the function using the formulae provided above
    pass


### uv normalisation matrix

The normalisation matrix for uv coordinates $\mathbf{T}_{\text{norm}}$ is defined as $$\mathbf{T}_{\text{norm}} = \begin{bmatrix}w + h & 0 & \frac{w}{2}\\[1ex]  0 & w + h & \frac{h}{2}\\[1ex] 0 & 0 & 1\\\end{bmatrix}^{-1}$$

In [ ]:
def uv_normalisation_mat(h, w):
    # TODO: Complete the function using the formulae provided above
    pass


### Calibrate

Function to perform camera calibration

**Usage:**   

```calibrate(img, xyz, uv)```

```return p_mat```

**Where:**  
 - img: the image of the calibration target.
 - xyz: a N x 3 array of  XYZ coordinates of the calibration target points. 
 - uv: a N x 2 array of the image coordinates of the calibration target points.
 - p_mat: the 3 x 4 camera calibration matrix.


The variable N should be an integer greater than or equal to 6.

**The root mean squared error (RMSE) between the positions of the uv coordinates and the projected XYZ coordinates should be reported.**


In [ ]:
def calibrate(img, xyz, uv):
    # pre-condition checking
    assert xyz.shape[0] >= 6 and xyz.shape[1] == 3
    assert uv.shape[0] >= 6 and uv.shape[1] == 2
    assert xyz.shape[0] == uv.shape[0]

    # TODO: Complete the function using the normalised DLT algorithm and print out the RMSE for reprojection
    p_mat = None
    return p_mat


### Calibrate the camera and decompose the calibration matrix with util functions

1. Obtain the camera calibration matrix with the `calibrate` function defined above
2. Decompose the $\mathbf{P}$ matrix into $\mathbf{K}$, $\mathbf{R}$, $\mathbf{t}$ using the util function defined above, such that $\mathbf{P} = \mathbf{K}[\mathbf{R}|\mathbf{t}]$.

In [ ]:
p_mat = calibrate(img, xyz, uv)
k_mat, r_mat, t = None, None, None

print(f"P: \n {p_mat}")
print(f"K: \n {k_mat}")
print(f"R: \n {r_mat}")
print(f"t: \n {t}")


### Discussion questions
Use the results from above to answer the following questions
1. What is the focal length (in the unit of a pixel) of the camera?
2. What is the camera centre coordinate in the XYZ coordinate system (world coordinate system)?

<b><font color="blue">Write your answers below:</font></b>
1. <font color="blue">Your solution to question 1</font></b>
2. <font color="blue">Your solution to question 2</font></b>